## Getting started

In this quickstart we will see how to:
1. Get set up with LangChain, LangSmith and LangServe
2. Use the most basic and common components of LangChain: prompt templates, models and output parsers
3. Build a simple application with Langchain
4. Trace the application with LangSmith
5. Serve the application with LangServe

In [1]:
import os
from dotenv import load_dotenv

# Load the environment variables in .env file
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
print(f'OPENAI_API_KEY: {OPENAI_API_KEY[:10]}***{OPENAI_API_KEY[-3:]}')

# We will use LANGCHAIN_API_KEY for LangSmith tracking
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
print(f'LANGCHAIN_API_KEY: {LANGCHAIN_API_KEY[:10]}***{LANGCHAIN_API_KEY[-3:]}')

# Required by LangChain 
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
LANGCHAIN_PROJECT = os.environ['LANGCHAIN_PROJECT']
print(f'LANGCHAIN_PROJECT: {LANGCHAIN_PROJECT}')

OPENAI_API_KEY: sk-proj-ID***tIA
LANGCHAIN_API_KEY: lsv2_pt_64***43b
LANGCHAIN_PROJECT: LangChain tutorial get started


One of the questions when we work with LLMs is, what kind of models we can actually use?

For OpenAI we can find the models in [OpenAI docs](http://platform.openai.com/docs/models)

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o', api_key=OPENAI_API_KEY)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x1175192b0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117519d30> root_client=<openai.OpenAI object at 0x116956900> root_async_client=<openai.AsyncOpenAI object at 0x117519a90> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


Asking a question. We need to provide **input** and we will get a **response** from the LLM. Because we have a `LANGCHAIN_API_KEY` set and also `LANGCHAIN_TRACING_V2` set to **true**, all the requests/responses will be stored in [LangSmith](http://smith.langchain.com/o/a9402f23-0cd4-4009-8f20-15b305f46b4f/projects/p/c2b7c2c3-ca78-46bf-bfc0-fdee6a119ba9?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=4b5d482e-cbc8-470b-b8dc-8f1162ef4952&peeked_trace=4b5d482e-cbc8-470b-b8dc-8f1162ef4952)

In [3]:
input = 'What is GenAI?'

response = llm.invoke(input=input)
response

AIMessage(content="GenAI, short for Generative Artificial Intelligence, refers to a subset of artificial intelligence that focuses on creating or generating new content. This can include text, images, music, and more, based on input data or prompts. GenAI models, such as OpenAI's GPT-3, DALL-E, and Google's Imagen, utilize deep learning techniques, particularly neural networks, to produce outputs that mimic human creativity or resemble real-world data.\n\nThese models are trained on large datasets and can understand and produce human-like language, create visuals that resemble photographs or artwork, compose music, and even code. The potential applications of GenAI are vast, impacting industries such as entertainment, design, marketing, and even software development. However, they also raise ethical and societal questions regarding originality, ownership, and the potential for misuse.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 163, 'p

### Prompt template

**Prompt template** is how you want the LLM to behave or what kind of role you want your LLM to be. 
Example: Act as an AI engineer and provide me suggestions how to develop AI project.


**ChatPromptTemplate** 
When we create a ChatPromptTemplate we define the prompt as a list of tuples. Each tuple 

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    # System
    ('system', 'You are expert AI engineer. Provide me answers based on the question'),
    # User. User will provide some input with {input} placeholder
    ('user', '{input}')
])
print(prompt)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are expert AI engineer. Provide me answers based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


Now let's use this **prompt template** along with the LLM model. For that we will create a **chain**. Chain is create like `chain = prompt|llm`. So whenever we give any input to the chain:
- It will go to the prompt 
- Go to the LLM
- Return the response

We are calling the chain with `invoke()` method and we need to pass all the input variables from the prompt as a `dict`.

In [5]:
chain = prompt | llm
response = chain.invoke({'input', 'What is LangSmith?'})
print(response)

content='LangSmith is a platform or tool designed to enhance and optimize the development, testing, and monitoring of applications built using large language models (LLMs). It aims to address the challenges developers face when working with LLM-based applications by offering features for debugging, testing, evaluating performance, and integrating seamlessly into existing workflows. LangSmith typically supports applications built with popular frameworks like LangChain, providing functionalities that help ensure the cutting-edge performance, reliability, and efficiency of language model implementations.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 33, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fin

#### Output parser
Output parser get the message from LLM and transforms it How we want to display it. 

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

response = chain.invoke({'input', 'What is the difference between SVD and PCA?'})
print(response)

Singular Value Decomposition (SVD) and Principal Component Analysis (PCA) are both techniques used in data analysis and dimensionality reduction, but they have different applications and interpretations:

1. **Purpose and Application**:
   - **SVD**: It is a mathematical technique used to factorize a matrix into three matrices (U, Σ, V*). SVD is a more general method and can be applied to any real or complex matrix. It's widely used not only in dimensionality reduction but also in signal processing, image compression, and solving systems of linear equations.
   - **PCA**: This is a specific statistical technique used to identify the directions (principal components) that maximize the variance in a dataset. PCA is primarily used for dimensionality reduction and exploratory data analysis to simplify the complexity of the data while retaining as much variance as possible.

2. **Mathematical Relationship**:
   - PCA is essentially a specific application of SVD. When you perform PCA, you ty

fr